In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime, timedelta

# ✅ Yahoo Finance Stock Market News 페이지 URL
URL = "https://finance.yahoo.com/topic/stock-market-news/"

# ✅ Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저 창 숨김
options.add_argument("--no-sandbox")  # 충돌 방지
options.add_argument("--disable-dev-shm-usage")  # 메모리 문제 해결
options.add_argument("--disable-gpu")  # GPU 가속 비활성화
options.add_argument("user-agent=Mozilla/5.0")

# ✅ WebDriver 실행
driver = webdriver.Chrome(options=options)
driver.get(URL)

# ✅ 페이지 완전 로드 대기
time.sleep(10)

# ✅ 중복 방지를 위한 URL 저장
seen_links = set()
news_data = []

# ✅ 기사 목록 가져오기 (최대 5개)
try:
    articles = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located(
            (By.XPATH, '//div[contains(@class, "topic-stories")]//a[contains(@href, "/news/")]')
        )
    )

    for article in articles:
        link = article.get_attribute("href").strip()

        # ✅ 중복된 기사 URL 스킵
        if link in seen_links:
            continue
        seen_links.add(link)

        # 새 창에서 기사 상세 페이지 크롤링
        driver.execute_script("window.open(arguments[0]);", link)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(5)

        # ✅ 기사 제목 (cover-title)
        title = "Unknown"
        try:
            title_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "cover-title"))
            )
            title = title_element.text.strip()
        except:
            pass

        # ✅ 언론사 (출처)
        publisher = "Unknown"
        try:
            publisher_element = driver.find_elements(By.XPATH, '//a[@class="subtle-link fin-size-small yf-1xqzjha"]')
            if publisher_element:
                publisher = publisher_element[0].get_attribute("title").strip()
        except:
            pass

        # ✅ 날짜 크롤링 및 변환
        us_date, ko_date = "Unknown", "Unknown"
        try:
            time_element = driver.find_elements(By.CLASS_NAME, "byline-attr-meta-time")
            if time_element:
                iso_timestamp = time_element[0].get_attribute("datetime")
                dt_obj = datetime.strptime(iso_timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
                us_date = dt_obj.strftime('%Y-%m-%d %H:%M')  # ✅ UTC 시간
                ko_date = (dt_obj + timedelta(hours=9)).strftime('%Y-%m-%d %H:%M')  # ✅ 한국 시간
        except:
            pass

        # ✅ 기사 본문 (atoms-wrapper 내부 p 태그)
        content = "Unknown"
        try:
            paragraphs = driver.find_elements(By.XPATH, '//div[@class="atoms-wrapper"]//p')
            if paragraphs:
                content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip()])
        except:
            pass

        # ✅ 데이터 저장
        news_data.append({
            "Title": title,
            "URL": link,
            "Publisher": publisher,
            "US_Date": us_date,
            "KO_Date": ko_date,
            "Content": content
        })

        # 새 창 닫고 원래 창으로 복귀
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        # ✅ 5개 기사까지만 저장
        if len(news_data) >= 5:
            break

except Exception as e:
    print(f"❌ XPath 문제 발생: {e}")

# ✅ DataFrame 생성 후 저장
df = pd.DataFrame(news_data)
if df.empty:
    print("❌ 크롤링된 데이터가 없습니다. 다시 확인하세요.")
else:
    df.to_csv("yahoo_finance_news.csv", index=False, encoding="utf-8")
    print("✅ Yahoo Finance 뉴스 크롤링 완료! ‘yahoo_finance_news.csv’ 파일 저장됨.")

# ✅ WebDriver 종료
driver.quit()